In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, Dataset
import json
from tqdm import tqdm
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np

Uncomment to try other questions, or insert your own context/question.

In [2]:
context="It is often recommended that before jumping into the Chinese market, exporters should invest in targeted market research and taste testing.\
In a country as large and diverse in tastes as China, it is critical to identify a specific target group of consumers and confirm if your product appeals to this group.\
An increase in retail space and consumer desire for convenience has provided a demand for frozen and microwaveable food, which will likely continue to grow."

In [3]:
#question = "What should exporters invest in?"
question = "What has to be identified and confirmed?"
#question = "What has provided a demand for frozen and microwaveable food?"

In [4]:
def get_prediction(context, question):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
     #loading tokenizer and model for extractive question answering
    fine_tuned_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/q-a-model/local_fine_tuned_roberta_on_newsqa')
    fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained('/kaggle/input/q-a-model/local_fine_tuned_roberta_on_newsqa')
    fine_tuned_model.to(device)
    
    inputs = fine_tuned_tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
    outputs = fine_tuned_model(**inputs)

    answer_start = torch.argmax(outputs[0])
    answer_end = torch.argmax(outputs[1]) + 1

    answer = fine_tuned_tokenizer.convert_tokens_to_string(fine_tuned_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer

def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ' '

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

def training_sequences(tokenizer, m_length, data):
    seq = tokenizer.texts_to_sequences(data)
    seq = pad_sequences(seq, maxlen = m_length, padding='post',truncating='post')
    return seq

In [5]:
def main_fun(context,question):
    
    #getting the answer from the context
    answer=get_prediction(context, question)
    
    answer=[answer]
    
    #loading translation model
    model=load_model('/kaggle/input/eng-to-fr-translate/translation_model.h5')
    
    #loading the english and french tokenizers
    with open('/kaggle/input/eng-to-fr-translate/eng_tokenizer.pickle', 'rb') as handle:
        english_tokenizer = pickle.load(handle)
    with open('/kaggle/input/eng-to-fr-translate/fr_tokenizer.pickle', 'rb') as handle:
        french_tokenizer = pickle.load(handle)
        
    #preprocessing the answer for translation
    preprocess_ans = training_sequences(english_tokenizer,None,answer)
    tmp_ans = pad_sequences(preprocess_ans, maxlen=453, padding='post',truncating='post')
    tmp_ans = tmp_ans.reshape((-1, 453))
    
    #getting the translation
    prediction= [logits_to_text(model.predict(tmp_ans[[0]])[0], french_tokenizer)]

    #printing results
    print("French Answer:\n")
    print(prediction[0])
    print("\n")
    print("English Answer:\n")
    print(answer)


In [6]:
main_fun(context,question)

1/1 [==============================] - 1s 1s/step
French Answer:

si votre sa le produit de groupe de                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          


Englis